# RandomVector

## Résumé

Dans ce segment, nous présentons la classe `RandomVector`. Nous présentons en particulier le lien et les différences avec la classe `ComposedDistribution`. 

## Références

* http://openturns.github.io/openturns/master/user_manual/_generated/openturns.RandomVector.html
* http://openturns.github.io/openturns/master/user_manual/_generated/openturns.ComposedDistribution.html
* http://openturns.github.io/openturns/master/user_manual/_generated/openturns.CompositeDistribution.html
* http://openturns.github.io/openturns/master/examples/probabilistic_modeling/composite_distribution.html

## Introduction

OpenTURNS fournit deux classes « associées » aux fonctions

* `ComposedDistribution` : une distribution multidimensionnelle définie par des marginales et une copule
* `RandomVector` : un vecteur aléatoire

On peut créer un `RandomVector` de deux manières :

* à partir d’une `Distribution`
* l’application d’une fonction G à un `RandomVector`

*Objectifs* :

* Comprendre le lien entre un `RandomVector` et une fonction
* Comprendre le rôle de la classe `RandomVector`

In [1]:
import openturns as ot

Une `ComposedDistribution` est composée de :
* des marginales
* une copule (par défaut, c’est la copule indépendante)

Exemple : on considère trois variables gaussiennes indépendantes et on souhaite créer la distribution associée. 

On commence par créer les marginales.

In [2]:
X0 = ot.Normal (0. ,1.)
X1 = ot.Normal (0. ,1.)
X2 = ot.Normal (0. ,1.)

Puis on crée la loi jointe.

In [3]:
inputDistribution = ot.ComposedDistribution ( ( X0 , X1 , X2 ))

Pour créer un `RandomVector` à partir d’une distribution on utilise le constructeur suivant :

In [4]:
inputRandomVector = ot.RandomVector ( inputDistribution )

Maintenant, on va pouvoir lui appliquer une fonction G.


Pour créer un `RandomVector` à partir d’une fonction G, il faut :
* un `RandomVector` en entrée
* une `Function` à appliquer sur l’entrée


Dans l'exemple suivant, on crée un `RandomVector` à partir d’un `RandomVector` et d’une fonction
G. On commence par créer une fonction avec l'opérateur Python `def`. 

In [5]:
def mySimulator (x):
    y0=x [0] + x[1] + x[2]
    y1=x[0] - x[1] * x[2]
    y=[y0 ,y1]
    return y

Puis on utilise les classe `PythonFunction` et `RandomVector` pour créer le vecteur de sortie.

In [6]:
myfunction = ot.PythonFunction (3 ,2 , mySimulator )
outputRandomVector = ot.CompositeRandomVector ( myfunction , inputRandomVector )

## Points communs et différences entre `RandomVector` et `ComposedDistribution`

Points communs entre `RandomVector` et `ComposedDistribution`

* la méthode getSample est commune aux deux classes.

Différence entre `RandomVector` et `ComposedDistribution` :
    
* un `RandomVector` n’a pas de méthode pour évaluer sa PDF. En
général, il faudrait avoir toute la distribution de Y=G(X) pour l’avoir.
En général, lorsque G est un code de calcul externe de type boîte
noire, c’est très coûteux, voire impossible.
* une `ComposedDistribution` dispose des méthodes pour évaluer la
PDF, la CDF, les quantiles, etc... En d’autres termes, toute la
distribution est connue. Exemple : la méthode `computeCDF`.

## Deux cas particuliers de `RandomVector`

* Cas 1 : Le `RandomVector` d’entrée a été construit à partir d’une
`ComposedDistribution` qu’on peut récupérer grâce à la méthode
`getDistribution`.


In [7]:
inputRandomVector.getDistribution ()

class=ComposedDistribution name=ComposedDistribution dimension=3 copula=class=IndependentCopula name=IndependentCopula dimension=3 marginal[0]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[2]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1]

* Cas 2 : Le `RandomVector` de sortie a été construit à partir d'une fonction : la méthode `getDistribution` échoue, et c’est normal.

In [8]:
# Génère une exception
# outputRandomVector.getDistribution ()

## Exercices

### Exercice 1 : une fonction à trois entrées

Définir la fonction myAnalyticalSimulator comme une `SymbolicFunction` implémentant la fonction suivante :
$$
\begin{eqnarray}
Y_1 &=& x_1 + x_2 + x_3 \\
Y_2 &=& x_1 - x_2 x_3
\end{eqnarray}
$$
Comment créer le `RandomVector` associé ?

### Exercice 2 : quatre conversions

Expérimenter les quatre conversions présentées ci-dessous :`
1. `RandomVector` → `RandomVector`
1. `Distribution` → `Distribution`
1. `Distribution` → `RandomVector`
1. `RandomVector` → `Distribution`

**Questions**
* Quelles sont les conversions possibles ?
* Pourquoi certaines conversions sont elles impossibles ?

### Exercice 3 : composition de RandomVector

On considère la fonction `mySimulator2`.

In [9]:
def mySimulator2 (x):
    y0=x [0]+ x [1]
    y1=x [1]* x [2]
    y=[y0 ,y1]
    return y

Utiliser la classe `RandomVector` pour définir le vecteur aléatoire associé à la composition de la fonction `mySimulator` par la fonction `mySimulator`, c’est à dire `Y=mySimulator2(mySimulator(X))`.

## Exercice 4 :  la classe CompositeDistribution

La classe `CompositeDistribution` permet de définir une distribution fondée sur l'application d'une fonction scalaire (de $\mathbb{R}$ dans $\mathbb{R}$) à une distribution unidimensionnelle. 
Considérons $X$ une variable aléatoire de loi $\mathcal{L}_X$ et $G : \mathbb{R} \rightarrow \mathbb{R}$ une fonction. On considère la variable aléatoire $Y$ définie par :
$$
Y=G(X).
$$
On note $\mathcal{L}_Y$ la distribution de la variable $Y$ : la classe `CompositeDistribution` permet de définir la loi de $Y$. 

On considère la variable $X$ de loi uniforme entre 0 et 12. On considère la fonction $G$ définie par :
$$
G(X) = X^2
$$
pour $X\in[0,12]$.

**Questions**
* Définir la variable `distributionX` associée à la loi de la variable $X$. Dessiner la densité de probabilité de la distribution.
* Définir la fonction `maFonc` associée à la fonction $G$. Dessiner la fonction entre 0 et 12.
* Utiliser la classe `CompositeDistribution` pour définir la distribution associée à $Y$. Dessiner cette distribution.